# 🧩 Step 1: Load Base Sales and Inventory Data
This step imports the required Python libraries (Pandas, NumPy) and loads base **sales** and **inventory** datasets.  
It simulates cleaned data from previous ETL tasks.


In [2]:
# Cell 1: imports and load base data
import pandas as pd
import numpy as np

# --- 1. LOAD CLEANED SALES AND INVENTORY DATA ---
sales_data_base = {
    'product_id': [101, 102, 101, 103, 102, 104],
    'product_name': ['Mouse', 'Coffee Maker', 'Mouse', 'Cable', 'Coffee Maker', 'Lamp'],
    'quantity_sold': [10, 5, 15, 20, 8, 3],
    'total_amount': [250.00, 275.00, 375.00, 100.00, 440.00, 90.00],
    'sale_date': pd.to_datetime(['2023-10-01', '2023-10-15', '2023-10-20', '2023-10-25', '2023-10-28', '2023-10-30'])
}
df_sales = pd.DataFrame(sales_data_base)

inventory_data = {
    'product_id': [101, 102, 103, 104],
    'current_stock': [40, 25, 10, 17],
    'cost_of_goods': [15.00, 30.00, 2.00, 18.00]
}
df_inventory = pd.DataFrame(inventory_data)

print("--- DataFrames Loaded Successfully ---")
print("Sales:")
print(df_sales)
print("\nInventory:")
print(df_inventory)


--- DataFrames Loaded Successfully ---
Sales:
   product_id  product_name  quantity_sold  total_amount  sale_date
0         101         Mouse             10         250.0 2023-10-01
1         102  Coffee Maker              5         275.0 2023-10-15
2         101         Mouse             15         375.0 2023-10-20
3         103         Cable             20         100.0 2023-10-25
4         102  Coffee Maker              8         440.0 2023-10-28
5         104          Lamp              3          90.0 2023-10-30

Inventory:
   product_id  current_stock  cost_of_goods
0         101             40           15.0
1         102             25           30.0
2         103             10            2.0
3         104             17           18.0


# 🔄 Step 2: Simulate ETL Process (Append & Inventory Update)
This step simulates an **incremental ETL update**:
- Appends new sales records to the main sales dataset.
- Updates inventory stock based on sales quantity.
- Adds new products if not already in inventory.


In [3]:
# Cell 2: simulate new batch and ETL merge/append + inventory updates
# --- 2. SIMULATE ETL PIPELINE (INCREMENTAL UPDATE/MERGE) ---
new_sales_data = {
    'product_id': [101, 105, 102],
    'product_name': ['Mouse', 'Webcam', 'Coffee Maker'],
    'quantity_sold': [5, 10, 2],
    'total_amount': [125.00, 400.00, 110.00],
    'sale_date': pd.to_datetime(['2023-11-01', '2023-11-02', '2023-11-03'])
}
df_new_sales = pd.DataFrame(new_sales_data)

# ETL Step 1: Append new data to the target sales table (Simulating Delta Lake APPEND/MERGE)
df_updated_sales = pd.concat([df_sales, df_new_sales], ignore_index=True)

# ETL Step 2: Update Inventory Stock (Transactional Update)
for index, row in df_new_sales.iterrows():
    p_id = row['product_id']
    qty = row['quantity_sold']

    if p_id in df_inventory['product_id'].values:
        df_inventory.loc[df_inventory['product_id'] == p_id, 'current_stock'] -= qty
    else:
        new_row = pd.DataFrame({
            'product_id': [p_id],
            'current_stock': [0 - qty],  # initial negative stock after sale
            'cost_of_goods': [40.00]     # assumed cost for new product
        })
        df_inventory = pd.concat([df_inventory, new_row], ignore_index=True)

print("\n--- ETL Steps Completed ---")
print(f"Total Sales Records (Base: {len(df_sales)}, New: {len(df_new_sales)}): {len(df_updated_sales)}")
print("\nUpdated Sales (sample):")
print(df_updated_sales)
print("\nUpdated Inventory:")
print(df_inventory)



--- ETL Steps Completed ---
Total Sales Records (Base: 6, New: 3): 9

Updated Sales (sample):
   product_id  product_name  quantity_sold  total_amount  sale_date
0         101         Mouse             10         250.0 2023-10-01
1         102  Coffee Maker              5         275.0 2023-10-15
2         101         Mouse             15         375.0 2023-10-20
3         103         Cable             20         100.0 2023-10-25
4         102  Coffee Maker              8         440.0 2023-10-28
5         104          Lamp              3          90.0 2023-10-30
6         101         Mouse              5         125.0 2023-11-01
7         105        Webcam             10         400.0 2023-11-02
8         102  Coffee Maker              2         110.0 2023-11-03

Updated Inventory:
   product_id  current_stock  cost_of_goods
0         101             35           15.0
1         102             23           30.0
2         103             10            2.0
3         104             17 

# 💾 Step 3: Save Updated Data as CSV
This step saves the updated **sales** and **inventory** data to CSV files.  
You can download them locally or save directly to Google Drive.


In [4]:
# Cell 3a: save to Colab filesystem and download
output_sales_file = 'updated_sales_data.csv'
output_inventory_file = 'updated_inventory_data.csv'

df_updated_sales.to_csv(output_sales_file, index=False)
df_inventory.to_csv(output_inventory_file, index=False)

print(f"Saved: {output_sales_file}, {output_inventory_file}")

# Download to your machine
from google.colab import files
files.download(output_sales_file)
files.download(output_inventory_file)


Saved: updated_sales_data.csv, updated_inventory_data.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# 📊 Step 4: SQL-Style Analysis (Top 5 Best-Selling Products)
This step performs a **SQL-style aggregation** using Pandas:
- Groups data by product name.
- Calculates total revenue and total units sold.
- Displays the **Top 5 best-selling products**.


In [5]:
# Cell 4: SQL-style aggregation to find top 5 best-selling products
top_5_products = df_updated_sales.groupby('product_name').agg(
    Total_Revenue=('total_amount', 'sum'),
    Total_Units_Sold=('quantity_sold', 'sum')
).reset_index().sort_values(by='Total_Revenue', ascending=False).head(5)

print("\n--- SQL-Style Analysis: Top 5 Best-Selling Products ---")
print(top_5_products)



--- SQL-Style Analysis: Top 5 Best-Selling Products ---
   product_name  Total_Revenue  Total_Units_Sold
1  Coffee Maker          825.0                15
3         Mouse          750.0                30
4        Webcam          400.0                10
0         Cable          100.0                20
2          Lamp           90.0                 3
